In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import gradio as gr

MODEL = "runwayml/stable-diffusion-v1-5"   # or another HF model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load pipeline (use use_safetensors=True if the model supports it)
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL, torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32
)
pipe = pipe.to(DEVICE)
pipe.enable_attention_slicing()  # reduces VRAM

def generate(prompt: str, height:int=512, width:int=512, num_images:int=1, guidance:float=7.5, steps:int=25, seed:int=None):
    generator = None
    if seed is not None:
        generator = torch.Generator(DEVICE).manual_seed(seed)
    images = pipe(prompt, height=height, width=width, num_inference_steps=steps,
                  guidance_scale=guidance, num_images_per_prompt=num_images, generator=generator).images
    return images  # list of PIL Images

# Quick Gradio UI
def gradio_generate(prompt, num_images, steps, guidance, seed):
    imgs = generate(prompt, num_images=int(num_images), steps=int(steps), guidance=float(guidance), seed=(None if seed=="" else int(seed)))
    return [img for img in imgs]

demo = gr.Interface(
    fn=gradio_generate,
    inputs=[
        gr.Textbox(label="Prompt", lines=2, value="A colorful surreal landscape, studio lighting, high detail"),
        gr.Slider(1, 4, step=1, value=1, label="Number of images"),
        gr.Slider(5, 50, step=1, value=25, label="DDIM / PLMS steps"),
        gr.Slider(1.0, 15.0, step=0.1, value=7.5, label="Guidance scale"),
        gr.Textbox(label="Seed (optional)", placeholder="empty for random"),
    ],
    outputs=gr.Gallery(label="Generated artworks"),
    title="AI-Powered Art Creator (Stable Diffusion)",
    description="Generate artworks from text prompts using Stable Diffusion."
)

demo.launch(share=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a962202cd148ecafff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
